In [23]:
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.models import load_model
import opticspy
import random
from random import randrange

In [24]:
parent_dir = ''
# for using on local windows machine

In [25]:
# Simple scaling normalization
def zernike_gen(batch_size=16,sqr_grid_width=50):
    
    while True:

        train_coeff = np.transpose([[random.uniform(-2.,3.) for Z1 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z2 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z3 in range(batch_size)],
                                    [random.uniform(-9.,5.) for Z4 in range(batch_size)],
                                    [random.uniform(-4.,2.) for Z5 in range(batch_size)],
                                    [random.uniform(-3.,3.2) for Z6 in range(batch_size)],
                                    [random.uniform(-1.,1.4) for Z7 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z8 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z9 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z10 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z11 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z12 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z13 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z14 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z15 in range(batch_size)]])
        
#         Divide x and y by 49 to normalize between 0 and 1
        x = np.array([np.array([[i for i in range(sqr_grid_width)] for x in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)
        y = np.array([np.array([[y for i in range(sqr_grid_width)] for y in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)


        # z_grid to be used to create training data for dx and dy to create the gradient data
        z_grid_train = np.array([np.array(opticspy.zernike.Coefficient(Z1=train_coeff[z][0],
                            Z2=train_coeff[z][1], Z3=train_coeff[z][2], Z4=train_coeff[z][3], Z5=train_coeff[z][4],
                            Z6=train_coeff[z][5], Z7=train_coeff[z][6], Z8=train_coeff[z][7], Z9=train_coeff[z][8],
                            Z10=train_coeff[z][9], Z11=train_coeff[z][10], Z12=train_coeff[z][11], Z13=train_coeff[z][12],
                            Z14=train_coeff[z][13], Z15=train_coeff[z][14]).zernikematrix(l=sqr_grid_width))
                            for z in range(batch_size)])


        gradient_train = [np.gradient(z_grid_train[i]) for i in range(batch_size)]
        
#         Divide by 10 to keep numbers near the -1 to 1 range
        dx_train = np.array([np.array([x for x in gradient_train[i][0]]).flatten() for i in range(batch_size)])
        dy_train = np.array([np.array([y for y in gradient_train[i][1]]).flatten() for i in range(batch_size)])

        train_input = np.array([np.transpose([x[i], y[i], dx_train[i], dy_train[i]])
                      for i in range(batch_size)])

        yield train_input,train_coeff

In [26]:
def zernike_gen_ones(batch_size=16,sqr_grid_width=50):
    
    while True:

        train_coeff = np.transpose([[random.uniform(-1.,1.) for Z1 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z2 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z3 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z4 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z5 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z6 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z7 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z8 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z9 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z10 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z11 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z12 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z13 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z14 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z15 in range(batch_size)]])
        
#         Divide x and y by 49 to normalize between 0 and 1
        x = np.array([np.array([[i for i in range(sqr_grid_width)] for x in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)
        y = np.array([np.array([[y for i in range(sqr_grid_width)] for y in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)


        # z_grid to be used to create training data for dx and dy to create the gradient data
        z_grid_train = np.array([np.array(opticspy.zernike.Coefficient(Z1=train_coeff[z][0],
                            Z2=train_coeff[z][1], Z3=train_coeff[z][2], Z4=train_coeff[z][3], Z5=train_coeff[z][4],
                            Z6=train_coeff[z][5], Z7=train_coeff[z][6], Z8=train_coeff[z][7], Z9=train_coeff[z][8],
                            Z10=train_coeff[z][9], Z11=train_coeff[z][10], Z12=train_coeff[z][11], Z13=train_coeff[z][12],
                            Z14=train_coeff[z][13], Z15=train_coeff[z][14]).zernikematrix(l=sqr_grid_width))
                            for z in range(batch_size)])


        gradient_train = [np.gradient(z_grid_train[i]) for i in range(batch_size)]
        
        dx_train = np.array([np.array([x for x in gradient_train[i][0]]).flatten() for i in range(batch_size)])
        dy_train = np.array([np.array([y for y in gradient_train[i][1]]).flatten() for i in range(batch_size)])

        train_input = np.array([np.transpose([x[i], y[i], dx_train[i], dy_train[i]])
                      for i in range(batch_size)])

        yield train_input,train_coeff

In [52]:
def block_gen(min_blk=2,max_blk=10,sqr_grid_width=50):
    x_block_size = randrange(min_blk,max_blk+1) 
    x_block_start = randrange(sqr_grid_width-1-x_block_size) 
    x_block = np.array(range(x_block_start,x_block_start+x_block_size))/float(sqr_grid_width)

    y_block_size = randrange(min_blk,max_blk+1) 
    y_block_start = randrange(sqr_grid_width-1-y_block_size) 
    y_block = np.array(range(y_block_start,y_block_start+y_block_size))/float(sqr_grid_width)
    
    return x_block,y_block

y_mask = lambda y,batch,block: np.isin(y[batch][:,1],block)

x_mask = lambda x,batch,block: np.isin(x[batch][:,0],block)

def zernike_gen_block(batch_size=16,sqr_grid_width=50):
   
    while True:
        
        train_coeff = np.transpose([[random.uniform(-2.,3.) for Z1 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z2 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z3 in range(batch_size)],
                                    [random.uniform(-9.,5.) for Z4 in range(batch_size)],
                                    [random.uniform(-4.,2.) for Z5 in range(batch_size)],
                                    [random.uniform(-3.,3.2) for Z6 in range(batch_size)],
                                    [random.uniform(-1.,1.4) for Z7 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z8 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z9 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z10 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z11 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z12 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z13 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z14 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z15 in range(batch_size)]])


        x = np.array([np.array([[i for i in range(sqr_grid_width)] for x in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)

        y = np.array([np.array([[y for i in range(sqr_grid_width)] for y in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)


        # z_grid to be used to create training data for dx and dy to create the gradient data
        z_grid_train = np.array([np.array(opticspy.zernike.Coefficient(Z1=train_coeff[z][0],
                            Z2=train_coeff[z][1], Z3=train_coeff[z][2], Z4=train_coeff[z][3], Z5=train_coeff[z][4],
                            Z6=train_coeff[z][5], Z7=train_coeff[z][6], Z8=train_coeff[z][7], Z9=train_coeff[z][8],
                            Z10=train_coeff[z][9], Z11=train_coeff[z][10], Z12=train_coeff[z][11], Z13=train_coeff[z][12],
                            Z14=train_coeff[z][13], Z15=train_coeff[z][14]).zernikematrix(l=sqr_grid_width))
                            for z in range(batch_size)])


        gradient_train = [np.gradient(z_grid_train[i]) for i in range(batch_size)]

        dx_train = np.array([np.array([x for x in gradient_train[i][0]]).flatten() for i in range(batch_size)])
        dy_train = np.array([np.array([y for y in gradient_train[i][1]]).flatten() for i in range(batch_size)])

        train_input = np.array([np.transpose([x[i], y[i], dx_train[i], dy_train[i]])
                            for i in range(batch_size)])

    #     NOTE: every sample in batch has the same block removed with this set up?
    #     Generate random block
        x_block,y_block = block_gen(min_blk=2,max_blk=10,sqr_grid_width=sqr_grid_width)

    #     Remove random block
        for i in range(batch_size):
            train_input[i][(y_mask(train_input,i,y_block)) & (x_mask(train_input,i,x_block))] = 0


        yield train_input,train_coeff

In [28]:
# Create random matrix to add to dx and dy
def add_noise(original,batch_size=1,mu=0,sigma=.01):
    noise = [[random.gauss(mu, sigma) for x in range(len(original[0]))] for i in range(batch_size)]
    mod_data = original + noise
    return np.array(mod_data)

def zernike_gen_noise(batch_size=16,sqr_grid_width=50):
    
    while True:
        
        train_coeff = np.transpose([[random.uniform(-2.,3.) for Z1 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z2 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z3 in range(batch_size)],
                                    [random.uniform(-9.,5.) for Z4 in range(batch_size)],
                                    [random.uniform(-4.,2.) for Z5 in range(batch_size)],
                                    [random.uniform(-3.,3.2) for Z6 in range(batch_size)],
                                    [random.uniform(-1.,1.4) for Z7 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z8 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z9 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z10 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z11 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z12 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z13 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z14 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z15 in range(batch_size)]])

        x = np.array([np.array([[i for i in range(sqr_grid_width)] for x in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)
        y = np.array([np.array([[y for i in range(sqr_grid_width)] for y in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)


        # z_grid to be used to create training data for dx and dy to create the gradient data
        z_grid_train = np.array([np.array(opticspy.zernike.Coefficient(Z1=train_coeff[z][0],
                            Z2=train_coeff[z][1], Z3=train_coeff[z][2], Z4=train_coeff[z][3], Z5=train_coeff[z][4],
                            Z6=train_coeff[z][5], Z7=train_coeff[z][6], Z8=train_coeff[z][7], Z9=train_coeff[z][8],
                            Z10=train_coeff[z][9], Z11=train_coeff[z][10], Z12=train_coeff[z][11], Z13=train_coeff[z][12],
                            Z14=train_coeff[z][13], Z15=train_coeff[z][14]).zernikematrix(l=sqr_grid_width))
                            for z in range(batch_size)])


        gradient_train = [np.gradient(z_grid_train[i]) for i in range(batch_size)]

        dx_train = np.array([np.array([x for x in gradient_train[i][0]]).flatten() for i in range(batch_size)])
        dx_noise = add_noise(dx_train,batch_size=batch_size)

        dy_train = np.array([np.array([y for y in gradient_train[i][1]]).flatten() for i in range(batch_size)])
        dy_noise = add_noise(dy_train,batch_size=batch_size)

        train_input = np.array([np.transpose([x[i], y[i], dx_noise[i], dy_noise[i]])
                      for i in range(batch_size)])

        yield train_input,train_coeff

In [29]:
# Create random matrix to add to dx and dy and block region
def zernike_gen_blocknoise(batch_size=16,sqr_grid_width=50):
    
    while True:
        
        train_coeff = np.transpose([[random.uniform(-2.,3.) for Z1 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z2 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z3 in range(batch_size)],
                                    [random.uniform(-9.,5.) for Z4 in range(batch_size)],
                                    [random.uniform(-4.,2.) for Z5 in range(batch_size)],
                                    [random.uniform(-3.,3.2) for Z6 in range(batch_size)],
                                    [random.uniform(-1.,1.4) for Z7 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z8 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z9 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z10 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z11 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z12 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z13 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z14 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z15 in range(batch_size)]])

        x = np.array([np.array([[i for i in range(sqr_grid_width)] for x in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)
        y = np.array([np.array([[y for i in range(sqr_grid_width)] for y in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)


        # z_grid to be used to create training data for dx and dy to create the gradient data
        z_grid_train = np.array([np.array(opticspy.zernike.Coefficient(Z1=train_coeff[z][0],
                            Z2=train_coeff[z][1], Z3=train_coeff[z][2], Z4=train_coeff[z][3], Z5=train_coeff[z][4],
                            Z6=train_coeff[z][5], Z7=train_coeff[z][6], Z8=train_coeff[z][7], Z9=train_coeff[z][8],
                            Z10=train_coeff[z][9], Z11=train_coeff[z][10], Z12=train_coeff[z][11], Z13=train_coeff[z][12],
                            Z14=train_coeff[z][13], Z15=train_coeff[z][14]).zernikematrix(l=sqr_grid_width))
                            for z in range(batch_size)])


        gradient_train = [np.gradient(z_grid_train[i]) for i in range(batch_size)]

        dx_train = np.array([np.array([x for x in gradient_train[i][0]]).flatten() for i in range(batch_size)])
        dx_noise = add_noise(dx_train,batch_size=batch_size)

        dy_train = np.array([np.array([y for y in gradient_train[i][1]]).flatten() for i in range(batch_size)])
        dy_noise = add_noise(dy_train,batch_size=batch_size)

        train_input = np.array([np.transpose([x[i], y[i], dx_noise[i], dy_noise[i]])
                      for i in range(batch_size)])
        
#     NOTE: every sample in batch has the same block removed with this set up?
#     Generate random block
        x_block,y_block = block_gen(min_blk=2,max_blk=10,sqr_grid_width=sqr_grid_width)

#     Remove random block
        for i in range(batch_size):
            train_input[i][(y_mask(train_input,i,y_block)) & (x_mask(train_input,i,x_block))] = 0

        yield train_input,train_coeff

In [30]:
def eval_model_prefab(test_num,parent_dir,x,y):
    os.chdir(parent_dir + '.\\Test_{}'.format(test_num))
    test_model = load_model('model_test_{}.h5'.format(test_num))
#     test_model = load_model('epoch.20.hdf5')
    return test_model.evaluate(x,y)

In [31]:
def eval_model_gen(test_num,parent_dir,gen,steps=1000,verbose=0,name_append=''):
    os.chdir(parent_dir + '.\\Test_{}{}'.format(test_num,name_append))
    test_model = load_model('model_test_{}.h5'.format(test_num))
    return test_model,test_model.evaluate_generator(gen,steps=steps,verbose=verbose)

## BASELINE MODEL EVALUATION

### Evaluate Perfect Samples

In [32]:
z_gen = zernike_gen(batch_size=8)

In [33]:
test_model,score = eval_model_gen(6,parent_dir,z_gen,verbose=1,name_append='_Perfect')

1000/1000 [==============================] - 38s 38ms/step


In [34]:
print(test_model.metrics_names,'\n',score)

['loss', 'mean_absolute_error'] 
 [0.14063341136835517, 0.09204327921569347]


### Evaluate Blocked Region Samples

In [53]:
z_gen_block = zernike_gen_block(batch_size=8)

In [54]:
test_model_block,score_block = eval_model_gen(6,parent_dir,z_gen_block,verbose=1,name_append='_Perfect')

1000/1000 [==============================] - 33s 33ms/step


In [55]:
print(test_model_block.metrics_names,'\n',score_block)

['loss', 'mean_absolute_error'] 
 [0.14378102636523546, 0.12326099549606442]


### Evaluate Noisy Samples

In [38]:
z_gen_noise = zernike_gen_noise(batch_size=8)

In [39]:
test_model_noise,score_noise = eval_model_gen(6,parent_dir,z_gen_noise,verbose=1,name_append='_Perfect')

1000/1000 [==============================] - 59s 59ms/step


In [40]:
print(test_model_noise.metrics_names,'\n',score_noise)

['loss', 'mean_absolute_error'] 
 [0.1390327976681292, 0.09443452239781618]


### Evaluate Noisy Blocked Region Samples

In [56]:
z_gen_blocknoise = zernike_gen_blocknoise(batch_size=8)

In [57]:
test_model_blocknoise,score_blocknoise = eval_model_gen(6,parent_dir,z_gen_blocknoise,verbose=1,name_append='_Perfect')

1000/1000 [==============================] - 54s 54ms/step


In [58]:
print(test_model_blocknoise.metrics_names,'\n',score_blocknoise)

['loss', 'mean_absolute_error'] 
 [0.14284816898033023, 0.12319548914581538]


## BLOCKED AND NOISY MODEL EVALUATION (Continued Training)

### Evaluate Perfect Samples

In [44]:
test_model2,score2 = eval_model_gen(6,parent_dir,z_gen,verbose=1,name_append='_BlockNoise')

1000/1000 [==============================] - 38s 38ms/step


In [45]:
print(test_model2.metrics_names,'\n',score2)

['loss', 'mean_absolute_error'] 
 [0.1422210692409426, 0.1083152702189982]


### Evaluate Blocked Region Samples

In [59]:
test_model_block2,score_block2 = eval_model_gen(6,parent_dir,z_gen_block,verbose=1,name_append='_BlockNoise')

1000/1000 [==============================] - 34s 34ms/step


In [60]:
print(test_model_block2.metrics_names,'\n',score_block2)

['loss', 'mean_absolute_error'] 
 [0.14414166660420596, 0.12189801286533475]


### Evaluate Noisy Samples

In [48]:
test_model_noise2,score_noise2 = eval_model_gen(6,parent_dir,z_gen_noise,verbose=1,name_append='_BlockNoise')

1000/1000 [==============================] - 57s 57ms/step


In [49]:
print(test_model_noise2.metrics_names,'\n',score_noise2)

['loss', 'mean_absolute_error'] 
 [0.13888896559737623, 0.10882283739000559]


### Evaluate Noisy Blocked Region Samples

In [61]:
test_model_blocknoise2,score_blocknoise2 = eval_model_gen(6,parent_dir,z_gen_blocknoise,verbose=1,name_append='_BlockNoise')

1000/1000 [==============================] - 52s 52ms/step


In [62]:
print(test_model_blocknoise2.metrics_names,'\n',score_blocknoise2)

['loss', 'mean_absolute_error'] 
 [0.1424532379861921, 0.121871656909585]


## BLOCKED AND NOISY MODEL EVALUATION (Trained With Random Weight Initialization)

### Evaluate Perfect Samples

In [63]:
test_model3,score3 = eval_model_gen(6,parent_dir,z_gen,verbose=1,name_append='_scratch_blocknoise')

1000/1000 [==============================] - 35s 35ms/step


In [64]:
print(test_model3.metrics_names,'\n',score3)

['loss', 'mean_absolute_error'] 
 [0.13947648993507028, 0.10576432262361049]


### Evaluate Blocked Region Samples

In [65]:
test_model_block3,score_block3 = eval_model_gen(6,parent_dir,z_gen_block,verbose=1,name_append='_scratch_blocknoise')

1000/1000 [==============================] - 36s 36ms/step


In [66]:
print(test_model_block3.metrics_names,'\n',score_block3)

['loss', 'mean_absolute_error'] 
 [0.14239149259962142, 0.11346308523416519]


### Evaluate Noisy Samples

In [67]:
test_model_noise3,score_noise3 = eval_model_gen(6,parent_dir,z_gen_noise,verbose=1,name_append='_scratch_blocknoise')

1000/1000 [==============================] - 55s 55ms/step


In [68]:
print(test_model_noise3.metrics_names,'\n',score_noise3)

['loss', 'mean_absolute_error'] 
 [0.1429164920654148, 0.10787252231687307]


### Evaluate Noisy Blocked Region Samples

In [69]:
test_model_blocknoise3,score_blocknoise3 = eval_model_gen(6,parent_dir,z_gen_blocknoise,verbose=1,
                                                          name_append='_scratch_blocknoise')

1000/1000 [==============================] - 54s 54ms/step


In [70]:
print(test_model_blocknoise3.metrics_names,'\n',score_blocknoise3)

['loss', 'mean_absolute_error'] 
 [0.14215625939145685, 0.11339534016698599]
